In [13]:
import json
import numpy as np
import pandas as pd 
import spacy

In [14]:
with open('plots.txt') as text:
    plots = text.read().split('<EOS>')

with open('titles.txt') as text:
    titles = text.read().split('\n')
    
for i in range(len(plots)):
    plots[i] = plots[i].replace("\n", " ").replace("\'", "")
    
df = pd.DataFrame({'titles': titles, 'plots': plots})
df.head()
print(len(df))

112937


My approach to the task is extracting the subject words from the plots with the help of spaCy package and counting the pronouns in them such as 'she', 'he', 'it' and 'they'. The plots having higher number of 'she' in comparison to 'he', 'it' and 'they' are most probably the plots having a female protagonist.

In [15]:
nlp = spacy.load("en_core_web_sm")

In [16]:
subjects = []
for plot in plots:
    doc = nlp(plot)
    subject = ', '.join([tok.text.lower() for tok in doc if tok.dep_ == 'nsubj'])
    subjects.append(subject)

In [34]:
len(subjects)

112937

In [35]:
df['subjects'] = pd.Series(subjects).astype(str)

In [36]:
df['she_count'] = df['subjects'].apply(lambda x: x.split(', ').count('she'))
df['he_count'] = df['subjects'].apply(lambda x: x.split(', ').count('he'))
df['it_count'] = df['subjects'].apply(lambda x: x.split(', ').count('it'))
df['they_count'] = df['subjects'].apply(lambda x: x.split(', ').count('they'))

In [37]:
df['most_she']= np.where((df['she_count'] > df['he_count']) & (df['she_count'] > df['it_count']) & (df['she_count'] > df['they_count']), 1, 0)

In [38]:
female_protagonist = df[df['most_she'] == 1]

In [39]:
len(female_protagonist)

3948

In [40]:
female_protagonist.to_excel('she.xlsx')

This approach is able to catch 3948 plots possibly having the female protagonist. After a manual check in Excel, the results seemed pretty robust.